In [1]:
import pandas as pd
import numpy as np
from scipy import sparse

from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, StandardScaler #, OrdinalEncoder
#from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin


from xgboost import XGBClassifier
import xgboost as xgb

from datetime import datetime
from datetime import date

import re
import pickle

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

from nltk.corpus import stopwords                   #Stopwords corpus
from nltk.tokenize import word_tokenize

import textblob
from textblob import TextBlob

from string import punctuation
##from spellchecker import SpellChecker

from transforms_dev import *

import requests,json

import csv
from datetime import datetime

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno 11001] getaddrinfo failed>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


### read data

In [2]:
df_tmp = pd.read_csv('C:/Projects/FCSD/FNOP/old/Testdata_pilot_sample_v2.csv', engine='python')

In [3]:
len(set(df_tmp['target']))

121

In [4]:
df_tmp.columns

Index(['clm_key', 'cust_txt', 'mdl_yr', 'prt_num_causl_base_cd', 'tis_wsd',
       'milge', 'veh_line_cd', 'eng_cd', 'cust_conc_cd', 'part_desc',
       'target'],
      dtype='object')

In [6]:
X_test = df_tmp[['mdl_yr', 'veh_line_cd', 'eng_cd', 'tis_wsd', 'milge',  'cust_txt']]

In [7]:
X_test.head(5)

mdl_yr veh_line_cd eng_cd  tis_wsd    milge  \
0    2011        C/AU   C/H4        0    731.0   
1    2011        C/AU   C/H4       34  46819.0   
2    2011        C/AU   C/H4       19  30971.0   
3    2011        C/AU   C/H4       10  10000.0   
4    2011        C/AU   C/H4        3   4552.0   

                                            cust_txt  
0  TOWED IN....BATTERY LIGHT, A.B.S. LIGHT, RADIO...  
1       C/S VEHICLE WILL NOT START AT TIMES. ADVISE.  
2  BATTERY INDICATOR LIGHT STAYS ON (IN INSTRUMEN...  
3  C/S BATTERY LIGHT IS COMING ON AND OFF. ADVISE...  
4            CUSTOMER CONCERN BATTERY LIGHT COMES ON

In [8]:
y_test_part = list(df_tmp['target'])

### use the pickled model to score X_test

In [15]:
le_yb = pickle.load(open('C:/Projects/FCSD/FNOP/old/encoder_v0.pkl', 'rb'))
classifier = pickle.load(open('C:/Projects/FCSD/FNOP/old/classifier_v0.pkl', 'rb'))

In [16]:
## generate all the probabilities from xgboost model
probs = classifier.predict_proba(X_test)

df_prob = pd.DataFrame(probs, columns=le_yb.classes_)

(24426, 111)


C:\Users\JLUO51\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:522: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


In [17]:
## find the top3 parts
k = 3

# GET TOP K PREDICTIONS BY PROB - note these are just index
best_n = np.argsort(probs, axis=1)[:,-k:]

# GET CATEGORY OF PREDICTIONS
preds=[[le_yb.classes_[predicted_cat] for predicted_cat in prediction] for prediction in best_n]

preds=[item[::-1] for item in preds]

preds

df_top3 = pd.DataFrame(preds, columns=("Top1", "Top2", "Top3"))
df_top3.head()

Top1    Top2    Top3
0   10346  other1  other2
1  other1   11002  other2
2  other1  other2   10346
3   10346  other1  other2
4   10346  other1  other2

In [18]:
N = len(y_test_part)
N

1026631

In [19]:
n_correct = 0
for i in range(N):
    if (df_top3.iloc[i, 0]==y_test_part[i]) or (df_top3.iloc[i, 1]==y_test_part[i]) or (df_top3.iloc[i, 2]==y_test_part[i]):
        n_correct += 1
        
print(n_correct/N)

0.8631767402309106


### check the top1 accuracy

In [20]:
n_correct_1 = 0
for i in range(N):
    if df_top3.iloc[i, 0]==y_test_part[i]:
        n_correct_1 += 1
        
print(n_correct_1/N)

0.6721012710506501


### generate processed text

In [9]:
lookup_table = 'C:/Projects/FCSD/FNOP/old/Misspellings_Correction_Synonyms_Lookup_Table_Master.xlsx'

In [10]:
syn_df=pd.read_excel(lookup_table)

In [11]:
syn_df = syn_df[syn_df['Review'] != "D"]

In [12]:
syn_dict=dict(zip(syn_df.misspelled, syn_df.corrected))

In [13]:
def lemmatize_with_postag(sentence):
    sent = TextBlob(sentence)
    tag_dict = {"J":'a',
                       "N":'n',
                       "V":'v',
                       "R":'r'}
    words_and_tags = [(w, tag_dict.get(pos[0],'n')) for w, pos in sent.tags]
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)

In [14]:
def TextPreprocess(text):
    # Avoid mixed data type errors and apply str on all rows

    text = str(text).lower()

    #Remove HTML Tags
    text = re.sub('<[^>]*>', '', text)
    r1 = re.compile('<.*?>')
    text = re.sub(r1, ' ', text)
    # Remove any token with numbers and/or special characters

    r2 = re.compile(r'^[0-9\W\_]+$')
    text = re.sub(r2, ' ', text)

    #Removing Punctuations
    text = re.sub(r'[?|!|\'|"|#]',r'',text)
    text = re.sub(r'[.|,|)|(|\|/]',r' ',text)        

    # Tokenize all text data using nltk package

    text = word_tokenize(text)

    # Remove stop words
    stop = stopwords.words('english')
    stop1 = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','c/s','c/states','c/stated','cu', 'cs','c-s', 'cus','tomer', 'stomer', 'tate', 'cust',  'cust states', 'advise','nana','NaN','customer','states','rr','nan','wont','ck','check','pls','please', 'client','owner']

    # Combine default nltk stop list and custom stop list to generate a deduplicated list
    english_stops = list(set(stop+stop1))

    text = [item for item in text if item not in english_stops]

    # Remove special characters occuring as prefixes
    text = [re.sub(r'^[\W\_]+','',str(i)) for i in text]
    # Remove special characters occuring as suffixes

    text = [re.sub(r'[\W\_]+$','',str(i)) for i in text]

    # Replace special characters with spaces when occuring within the token 
    #(i.e. not at the beginning or the end) only for tokens greater than 5 characters in length

    text =[re.sub(r'[\W\_]+', ' ', str(i)) if len(str(i)) > 5 else i for i in text]

    # Replace special characters with null when occuring within the token 
    #(i.e. not at the beginning or the end) only for tokens less than 6 characters in length

    text = [re.sub(r'[\W\_]+', '', str(i)) if len(str(i)) < 6 else i for i in text]
    # Remove stop words after data cleansing steps to remove any residual stop words

    text = [item for item in text if item not in english_stops]

    # remove any numbers after data cleansing steps to remove any residual numeric values

    r3 = re.compile(r'^[0-9\W\_]+$')
    text = [i for i in text if not r3.match(i)]

    #Perform synonym lookup to find matching correctly spelled tokens for misspelled tokens
    #spell=SpellChecker() 

    text = [syn_dict.get(str(i), str(i)) for i in text]

    # Convert the list to string before passing to lemmatization module
    text = ' '.join(text)

    # Perform Lemmatization on the cleaned up data before tfidf vectorization

    text = lemmatize_with_postag(text)

    return(text)

In [21]:
X_test['processed_txt']=X_test['cust_txt'].apply(lambda x: TextPreprocess(x))

C:\Users\JLUO51\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [22]:
X_test[['processed_txt']].to_csv("C:/Projects/FCSD/FNOP/old/processed_cust_txt_testset_v2.csv", index=False)

In [25]:
X_test.shape

(1026631, 7)

In [191]:
## text_input_df = pd.read_csv("C:/Projects/FCSD/FNOP/old/processed_cust_txt_testset_v2.csv")

In [192]:
## text_input_df.head(4)

process_txt
0              noise engine area possible alternator
1  whine noise engine area ken verify drop last n...
2         battery light come drive whole way morning
3              tow battery light light radio driving

### combine reliability score column

In [29]:
reliability_df = pd.read_csv("C:/Projects/FCSD/FNOP/old/f_rate_WB_118month_poc3.csv", engine='python')

In [30]:
reliability_df.columns = ['prt_num', 'veh_line_cd', 'mdl_yr', 'tis_wsd', 'Avg_Weibull_Failure', 'f_rate']

In [31]:
data_reliab_pivot = reliability_df.pivot_table(index=['veh_line_cd', 'mdl_yr', 'tis_wsd'], columns='prt_num', values=['Avg_Weibull_Failure'], aggfunc='first')
        
reliability_df_t = data_reliab_pivot.fillna(0.0)

In [32]:
df1 = pd.merge(X_test, reliability_df_t, how='left', on=['veh_line_cd', 'mdl_yr', 'tis_wsd'])

C:\Users\JLUO51\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:522: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


### combine X_test with actual_label and predicted_label

In [33]:
df_output = pd.concat([df1, df_tmp[['target']], df_top3[['Top1']]], axis=1)

In [34]:
df_output.shape

(1026631, 120)

In [36]:
df_output.head(5)

mdl_yr veh_line_cd eng_cd  tis_wsd    milge  \
0    2011        C/AU   C/H4        0    731.0   
1    2011        C/AU   C/H4       34  46819.0   
2    2011        C/AU   C/H4       19  30971.0   
3    2011        C/AU   C/H4       10  10000.0   
4    2011        C/AU   C/H4        3   4552.0   

                                            cust_txt  \
0  TOWED IN....BATTERY LIGHT, A.B.S. LIGHT, RADIO...   
1       C/S VEHICLE WILL NOT START AT TIMES. ADVISE.   
2  BATTERY INDICATOR LIGHT STAYS ON (IN INSTRUMEN...   
3  C/S BATTERY LIGHT IS COMING ON AND OFF. ADVISE...   
4            CUSTOMER CONCERN BATTERY LIGHT COMES ON   

                                     processed_txt  \
0            tow battery light light radio driving   
1                               vehicle start time   
2  battery indicator light stay instrument cluster   
3                        battery light come loaner   
4                       concern battery light come   

   (Avg_Weibull_Failure, 10346)  (Avg_Weibull_Failure, 10370)  \
0                           NaN                           NaN   
1                      0.000838                           0.0   
2                      0.000505                           0.0   
3                      0.000289                           0.0   
4                      0.000101                           0.0   

   (Avg_Weibull_Failure, 10655)  ...  (Avg_Weibull_Failure, 67R)  \
0                           NaN  ...                         NaN   
1                      0.001633  ...                         0.0   
2                      0.000473  ...                         0.0   
3                      0.000121  ...                         0.0   
4                      0.000009  ...                         0.0   

   (Avg_Weibull_Failure, 7210)  (Avg_Weibull_Failure, 7251)  \
0                          NaN                          NaN   
1                          0.0                          0.0   
2                          0.0                          0.0   
3                          0.0                          0.0   
4                          0.0                          0.0   

   (Avg_Weibull_Failure, 7B155)  (Avg_Weibull_Failure, 90)  \
0                           NaN                        NaN   
1                           0.0                        0.0   
2                           0.0                        0.0   
3                           0.0                        0.0   
4                           0.0                        0.0   

   (Avg_Weibull_Failure, 90T5)  (Avg_Weibull_Failure, 94RH7)  \
0                          NaN                           NaN   
1                          0.0                           0.0   
2                          0.0                           0.0   
3                          0.0                           0.0   
4                          0.0                           0.0   

   (Avg_Weibull_Failure, 96R)  target    Top1  
0                         NaN   10346   10346  
1                    0.043409   10346  other1  
2                    0.010368   10346  other1  
3                    0.002106   10346   10346  
4                    0.000105   10346   10346  

[5 rows x 120 columns]

In [37]:
df_output.to_csv("C:/Projects/FCSD/FNOP/old/testset_with_predicted_label_with_reliability_score.csv", index=False)

In [38]:
df_output_1 = pd.concat([X_test, df_tmp[['target']], df_top3[['Top1']]], axis=1)

In [39]:
df_output_1.shape

(1026631, 9)

In [40]:
df_output_1.to_csv("C:/Projects/FCSD/FNOP/old/testset_with_predicted_label.csv", index=False)